### HW2 - Multinomial Logistic Regression & SVMs 

### Training and Evaluations

Connect gdrive for train/test datasets:

In [ ]:
from google.colab import drive # import lib
drive.mount("/content/gdrive") # mount gdrive

Mounted at /content/gdrive


Import libraries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from pandas import *
from sklearn.model_selection import StratifiedKFold
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

Read Data:

In [ ]:
sTrainAdd = "/content/gdrive/MyDrive/MsC/cmp712/hw2/train.csv" # address of train.csv on gdrive 
sTestAdd = "/content/gdrive/MyDrive/MsC/cmp712/hw2/test.csv" # address of train.csv on gdrive 
data_training = read_csv(sTrainAdd)
data_test = read_csv(sTestAdd)

Preprocess Data:

In [ ]:
# Training Data Preprocessing
temp = data_training.iloc[:,:]
nRows, nColumns = temp.shape
sClass = "type"
x_training = DataFrame()
y_training = DataFrame()
for idxColumns in range(nColumns):
  temp1 = data_training.iloc[:, idxColumns]
  if temp1._name==sClass:
    y_training = y_training.append(temp1)
  else:
    x_training = x_training.append(temp1)
x_training = x_training.T
y_training = y_training.T
# Test Data Preprocessing
temp = data_test.iloc[:,:]
nRows, nColumns = temp.shape
sClass = "type"
x_test = DataFrame()
y_test = DataFrame()
for idxColumns in range(nColumns):
  temp1 = data_test.iloc[:, idxColumns]
  if temp1._name==sClass:
    y_test = y_test.append(temp1)
  else:
    x_test = x_test.append(temp1)
x_test = x_test.T
y_test = y_test.T

Encode Datasets:

In [ ]:
# Fix Length Feature (remove 'm' in the end and convert string to float)
# Training:
temp = data_training.iloc[:,:]
nRows, nColumns = temp.shape
for idx in range(nRows):
  temp = x_training.length[idx]
  if isinstance(temp, str):
    x_training.length[idx] = float(temp[:-1])
  elif math.isnan(temp):
    x_training.length[idx] = 9999
# Test:
temp = data_test.iloc[:,:]
nRows, nColumns = temp.shape
for idx in range(nRows):
  temp = x_test.length[idx]
  if isinstance(temp, str):
    x_test.length[idx] = float(temp[:-1])
  elif math.isnan(temp):
    x_test.length[idx] = 9999

In [ ]:
# Encoding (Logistic Regression):
# Encoding for Training Dataset:
le = LabelEncoder()
x_training_LR = x_training
x_training_LR.diet = le.fit_transform(x_training_LR.diet)
x_training_LR.lived_in = le.fit_transform(x_training_LR.lived_in)
x_training_LR.period = le.fit_transform(x_training_LR.period)
x_training_LR.named_by = le.fit_transform(x_training_LR.named_by)
x_training_LR.taxonomy = le.fit_transform(x_training_LR.taxonomy)
x_training_LR.species = le.fit_transform(x_training_LR.species)
# Limit dataset, drop id, name and link as they will not be used in training #
x_training_LR = x_training_LR.drop(columns=['id', 'name', 'link']) # x_, argument used in training
print(x_training_LR)
# Encoding for Test Dataset:
x_test_LR = x_test
x_test_LR.diet = le.fit_transform(x_test_LR.diet)
x_test_LR.lived_in = le.fit_transform(x_test_LR.lived_in)
x_test_LR.period = le.fit_transform(x_test_LR.period)
x_test_LR.named_by = le.fit_transform(x_test_LR.named_by)
x_test_LR.taxonomy = le.fit_transform(x_test_LR.taxonomy)
x_test_LR.species = le.fit_transform(x_test_LR.species)
# Limit dataset, drop id, name and link as they will not be used in training #
x_test_LR = x_test_LR.drop(columns=['id', 'name', 'link']) # x_, argument used in training
print(x_test_LR)

     diet  period  lived_in length  taxonomy  named_by  species
0       0     108        25   9999         0        76       61
1       2     108         1   5.15        34        11       95
2       1      65         5   12.0        20       212      188
3       1      43        24   21.0        49        64      181
4       1      21        25   10.0        17        22       24
..    ...     ...       ...    ...       ...       ...      ...
241     1      11        13   9999        25       186      184
242     1      10         2    3.0        13       161      120
243     0      14         5    0.8        79       203      222
244     0      42         1    7.6        58        14      182
245     3     112        19   12.0        34        70      170

[246 rows x 7 columns]
    diet  period  lived_in length  taxonomy  named_by  species
0      1      18        13    5.0        24        36       19
1      1      41        17   23.0        21         8       51
2      0      24   

In [ ]:
# Logistic Regression #
# Training #
skf = StratifiedKFold(n_splits=5)  #the number of folds is 5
model = LogisticRegression(solver='lbfgs', max_iter=1000)
accuracy_scores = []
for train_index, test_index in skf.split(x_training_LR, y_training):
  X_train, X_test = x_training_LR.iloc[train_index], x_training_LR.iloc[test_index]
  y_train, y_test = y_training.iloc[train_index], y_training.iloc[test_index]  
  model.fit(X_train, y_train)
  accuracy = model.score(X_test, y_test)
  accuracy_scores.append(accuracy)
# Compute the mean accuracy across all folds
np.savetxt('training_LR.csv', model.predict(x_training_LR), delimiter=',', fmt='%s') # save result to text
np.savetxt('test_LR.csv', model.predict(x_test_LR), delimiter=',', fmt='%s') # save result to text
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)

In [ ]:
# Logistic Regression - F1 Score and Confusion Matrix
f1_scores_LR = []
confusion_matrix_LR = []
# Perform k-fold stratified cross-validation
for train_index, test_index in skf.split(x_training_LR, y_training):
    X_train, X_test = x_training_LR.iloc[train_index], x_training_LR.iloc[test_index]
    y_train, y_test = y_training.iloc[train_index], y_training.iloc[test_index] 
    model.fit(X_train, y_train) # Necessary code to compute the predictions using your classifier.
    y_pred = model.predict(X_test)
    # Compute the weighted-average F1-score for this fold
    fold_f1_score = f1_score(y_test, y_pred, average='weighted')
    f1_scores_LR.append(fold_f1_score)
    fold_confusion_matrix = confusion_matrix(y_test, y_pred)
    confusion_matrix_LR.append(fold_confusion_matrix)
# Calculate the mean F1-score across all folds
mean_weighted_f1_score = np.mean(f1_scores_LR)

In [ ]:
print("Logistic Regression - Mean accuracy:", mean_accuracy)
print("Logistic Regression - Mean weighted-average F1-score across", 5, "folds:", mean_weighted_f1_score)
print("Logistic Regression - Confusion Matrix", confusion_matrix_LR)

Logistic Regression - Mean accuracy: 0.5157551020408163
Logistic Regression - Mean weighted-average F1-score across 5 folds: 0.5050917807653164
Logistic Regression - Confusion Matrix [array([[1, 0, 2, 0, 2, 0],
       [0, 2, 3, 0, 0, 0],
       [1, 0, 6, 0, 2, 0],
       [0, 0, 0, 7, 1, 2],
       [0, 0, 1, 1, 9, 0],
       [1, 1, 0, 1, 0, 7]]), array([[1, 0, 3, 0, 1, 0],
       [0, 3, 1, 0, 0, 0],
       [2, 1, 4, 0, 2, 0],
       [0, 0, 1, 5, 1, 3],
       [2, 0, 2, 1, 3, 3],
       [0, 0, 0, 6, 1, 3]]), array([[1, 0, 2, 0, 2, 0],
       [0, 4, 0, 0, 0, 0],
       [1, 1, 7, 0, 0, 0],
       [0, 0, 0, 7, 1, 2],
       [0, 0, 0, 1, 6, 4],
       [0, 0, 0, 5, 1, 4]]), array([[1, 0, 1, 0, 4, 0],
       [0, 3, 1, 0, 0, 0],
       [0, 1, 6, 0, 3, 0],
       [0, 0, 0, 2, 2, 5],
       [0, 0, 0, 1, 9, 1],
       [0, 0, 0, 6, 2, 1]]), array([[1, 0, 1, 0, 3, 0],
       [0, 4, 1, 0, 0, 0],
       [2, 1, 6, 0, 1, 0],
       [0, 0, 0, 5, 2, 2],
       [1, 0, 1, 1, 6, 2],
       [0, 0, 0, 5, 1, 3]

SVM:

This re-reading dataset in following codeblock is due to my bad coding, i saw that using multilabel binarizer works good for SVM, but not for Log. Reg. (i don't know why).

In [ ]:
data_training = read_csv(sTrainAdd)
data_test = read_csv(sTestAdd)
# Training Data Preprocessing
temp = data_training.iloc[:,:]
nRows, nColumns = temp.shape
sClass = "type"
x_training = DataFrame()
y_training = DataFrame()
for idxColumns in range(nColumns):
  temp1 = data_training.iloc[:, idxColumns]
  if temp1._name==sClass:
    y_training = y_training.append(temp1)
  else:
    x_training = x_training.append(temp1)
x_training = x_training.T
y_training = y_training.T
# Test Data Preprocessing
temp = data_test.iloc[:,:]
nRows, nColumns = temp.shape
sClass = "type"
x_test = DataFrame()
y_test = DataFrame()
for idxColumns in range(nColumns):
  temp1 = data_test.iloc[:, idxColumns]
  if temp1._name==sClass:
    y_test = y_test.append(temp1)
  else:
    x_test = x_test.append(temp1)
x_test = x_test.T
y_test = y_test.T
# Fix Length Feature (remove 'm' in the end and convert string to float)
# Training:
temp = data_training.iloc[:,:]
nRows, nColumns = temp.shape
for idx in range(nRows):
  temp = x_training.length[idx]
  if isinstance(temp, str):
    x_training.length[idx] = float(temp[:-1])
  elif math.isnan(temp):
    x_training.length[idx] = 9999
# Test:
temp = data_test.iloc[:,:]
nRows, nColumns = temp.shape
for idx in range(nRows):
  temp = x_test.length[idx]
  if isinstance(temp, str):
    x_test.length[idx] = float(temp[:-1])
  elif math.isnan(temp):
    x_test.length[idx] = 9999
# Encoding (SVM):
# Encoding for Training Dataset:
le2 = LabelEncoder()
mle = MultiLabelBinarizer()
x_training_SVM = x_training
x_training_SVM.diet = le2.fit_transform(x_training_SVM.diet)
x_training_SVM.lived_in = le2.fit_transform(x_training_SVM.lived_in)
x_training_SVM.period = le2.fit_transform(x_training_SVM.period)
x_training_SVM.named_by = le2.fit_transform(x_training_SVM.named_by)

x_training_SVM.taxonomy = mle.fit_transform(x_training_SVM.taxonomy)
x_training_SVM.species = le2.fit_transform(x_training_SVM.species)
# Limit dataset, drop id, name and link as they will not be used in training #
x_training_SVM = x_training_SVM.drop(columns=['id', 'name', 'link']) # x_, argument used in training
# Encoding for Test Dataset:
x_test_SVM = x_test
x_test_SVM.diet = le2.fit_transform(x_test_SVM.diet)
x_test_SVM.lived_in = le2.fit_transform(x_test_SVM.lived_in)
x_test_SVM.period = le2.fit_transform(x_test_SVM.period)
x_test_SVM.named_by = le2.fit_transform(x_test_SVM.named_by)

x_test_SVM.taxonomy = mle.fit_transform(x_test_SVM.taxonomy)
x_test_SVM.species = le2.fit_transform(x_test_SVM.species)
# Limit dataset, drop id, name and link as they will not be used in training #
x_test_SVM = x_test_SVM.drop(columns=['id', 'name', 'link']) # x_, argument used in training

In [ ]:
# SVM #
# Training and mean_accuracy #
skf = StratifiedKFold(n_splits=3, shuffle=True)  # the number of folds is 3
model_SVM = SVC(kernel = 'linear', decision_function_shape='ovo')
accuracy_scores = []
for train_index, test_index in skf.split(x_training_SVM, y_training):
  X_train, X_test = x_training_SVM.iloc[train_index], x_training_SVM.iloc[test_index]
  y_train, y_test = y_training.iloc[train_index], y_training.iloc[test_index]
  model_SVM.fit(X_train, y_train)
  y_pred = model_SVM.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  accuracy_scores.append(accuracy)
# Compute the mean accuracy across all folds
np.savetxt('training_SVM.csv', model_SVM.predict(x_training_SVM), delimiter=',', fmt='%s') # save result to text
np.savetxt('test_SVM.csv', model_SVM.predict(x_test_SVM), delimiter=',', fmt='%s') # save result to text
mean_accuracy_SVM = sum(accuracy_scores) / len(accuracy_scores)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# Calculate SVM F1 score and Confusion Matrix
f1_scores_SVM = []
confusion_matrix_SVM = []
# Perform k-fold stratified cross-validation
for train_index, test_index in skf.split(x_training_SVM, y_training):
    X_train, X_test = x_training_SVM.iloc[train_index], x_training_SVM.iloc[test_index]
    y_train, y_test = y_training.iloc[train_index], y_training.iloc[test_index] 
    model_SVM.fit(X_train, y_train) # Necessary code to compute the predictions using your classifier.
    y_pred = model_SVM.predict(X_test)
    # Compute the weighted-average F1-score for this fold
    fold_f1_score_svm = f1_score(y_test, y_pred, average='weighted')
    f1_scores_SVM.append(fold_f1_score_svm)
    fold_confusion_matrix_SVM = confusion_matrix(y_test, y_pred)
    confusion_matrix_SVM.append(fold_confusion_matrix_SVM)
# Calculate the mean F1-score across all folds
mean_weighted_f1_score_SVM = np.mean(f1_scores_SVM)
print("SVM - Mean weighted-average F1-score across", 5, "folds:", mean_weighted_f1_score_SVM)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVM - Mean weighted-average F1-score across 5 folds: 0.3393613095202855


In [ ]:
print("SVM - Mean accuracy:", mean_accuracy_SVM)
print("SVM - Mean weighted-average F1-score across", 5, "folds:", mean_weighted_f1_score_SVM)
print("Logistic Regression - Confusion Matrix", confusion_matrix_SVM)

SVM - Mean accuracy: 0.4146341463414634
SVM - Mean weighted-average F1-score across 5 folds: 0.3393613095202855
Logistic Regression - Confusion Matrix [array([[ 0,  0,  6,  0,  3,  0],
       [ 0,  0,  5,  0,  2,  0],
       [ 0,  0, 12,  0,  3,  0],
       [ 0,  0,  1,  6,  2,  7],
       [ 0,  0,  7,  0, 12,  0],
       [ 0,  0,  0, 10,  4,  2]]), array([[ 0,  0,  6,  0,  1,  2],
       [ 1,  0,  2,  0,  0,  4],
       [ 3,  0,  2,  0,  3,  8],
       [ 0,  0,  0, 14,  2,  0],
       [ 1,  0, 11,  0,  5,  1],
       [ 0,  0,  0,  2,  0, 14]]), array([[ 0,  0,  5,  0,  3,  0],
       [ 0,  0,  5,  0,  3,  0],
       [ 0,  0, 11,  0,  5,  0],
       [ 0,  0,  2, 12,  2,  0],
       [ 0,  0,  8,  0, 10,  0],
       [ 0,  0,  1, 13,  2,  0]])]
